In [1]:
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
import numpy as np
import random
from SoccerWorld import player, game

In [2]:
def ce_action(qtable, player_state, rand_rate, num_actions, pi):

    if np.random.rand() < rand_rate:
        idx = np.random.choice(np.arange(25), 1)
        return np.array([idx // 5, idx % 5]).reshape(2)
    else:
#         print (np.sum(pi))
        idx = random.choices(np.arange(25), weights=pi)[0]
        return np.array([idx // 5, idx % 5]).reshape(2)

    return action


def lp(qtable1, qtable2, state):
    a = qtable1[state, :, :]
    b = qtable2[state, :, :]
    
    arr = np.zeros((a.shape[0]**2))
    z = np.zeros((a.shape[0]**2))
    for i in range(a.shape[0]):
        for j in range(a.shape[0]):
            if i != j:
                q = a[j] - a[i]
                n = i * (a.shape[0])
                arr[n:n+a.shape[0]] = q 
                z = np.vstack((z, arr))
                arr = np.zeros((a.shape[0]**2))
    # print (z)

    for i in range(b.shape[0]):
        for j in range(b.shape[0]):
            if i != j:
                l = np.transpose(np.array(range(a.shape[0]**2)).reshape(a.shape[0],a.shape[0])).reshape(a.shape[0]**2)  

                q = b[j] - b[i]
                n = i * (a.shape[0])
                for kk in range(q.shape[0]):
                    arr[l[kk]+i] = q[kk]

                z = np.vstack((z, arr))
                arr = np.zeros((a.shape[0]**2))
    G = np.delete(z, 0, axis=0)
    eye = np.eye(G.shape[1]) * -1
    G = np.vstack((G, eye))
    q = np.ones(a.shape[0]**2) * 1.
    G = np.vstack((G, q))
    q = np.ones(a.shape[0]**2) * -1.
    G = np.vstack((G, q))
    b = np.zeros((G.shape[0], 1))
    b[-2] = 1
    b[-1] = -1
#     print (G)
    c = np.zeros((a.shape[0] ** 2))
#     print (c)
#     print (b)
    c = matrix(c)    
    G = matrix(G)
    b = matrix(b)
    sol = solvers.lp(c,G,b, solver='glpk') 
    if sol['x'] is not None:
        probs = np.array(sol['x']).reshape(a.shape[0], a.shape[0])
        v1 = np.sum(probs * qtable1[state])
        v2 = np.sum(probs * qtable2[state])
        return probs, v1, v2
    else:
        return None, None, None

         
   
        

In [17]:
 
def ce_q_learning():
    
    errors = []
    qs = []
    #hyperparameters
    num_actions = 5
    num_rows = 2
    num_cols = 4
    num_states = num_rows * num_cols * 2 #2 is for possession
#     n = 1000000
    n = 1000000
    gamma = 0.9
    alpha = 0.7
    alpha_dec = 0.95
    rar = 0.5
    radr = 0.99
    rand_rate = rar
    qtableA = np.zeros(shape=(num_states, num_actions, num_actions), dtype='float')
    qtableB = np.zeros(shape=(num_states, num_actions, num_actions), dtype='float')
    verbose = False
    
    
    V1 = np.ones(num_states) * 1.
    V2 = np.ones(num_states) * 1.
    
    pi = np.ones(shape=(num_states, num_actions**2)) * 1./25
    
    i = 0
    errors.append(0)
    ind = []
    ind.append(0)
    qs.append(0)
    while i < n:
        match = game(verbose=verbose, rows=2, columns=4, goalRstart=0, \
                     goalRend=1, aGoal=0, bGoal=3,aPosition=[0,2],\
                     bPosition=[0,1])
        
        initial_state = match.reset()
        
        state, score, done = match.move(4, 4)
        
        
        rand_rate *= radr
        
        if rand_rate < 0.05:
            rand_rate = 0.05
            


        while not done:

            old_state = state
            pA_act = ce_action(qtableA, state[0], rand_rate, num_actions, pi[state[0]])[0]
            pB_act = ce_action(qtableB, state[1], rand_rate, num_actions, pi[state[1]])[0]
            state, score, done = match.move(pA_act, pB_act)

   
                
            qtableA[old_state[0], pA_act, pB_act] = (1-alpha) * qtableA[old_state[0], pA_act, pB_act] + alpha * (score[0] + gamma * V1[state[0]])
            qtableB[old_state[1], pB_act, pA_act] = (1-alpha) * qtableB[old_state[1], pB_act, pA_act] + alpha * (score[1] + gamma * V2[state[1]])        

            probs, val1, val2 = lp(qtableA, qtableB, old_state[0])
            if probs is not None:
                pi[old_state[0], :] = probs.reshape(25)
                V1[old_state[0]] = val1
                V2[old_state[1]] = val2

            
            if old_state[0] == 2 and pA_act == 1 and pB_act == 4:
                    
                error = abs(qtableA[2, 1, 4] - qs[-1])
                qs.append(qtableA[2,1,4])
                print (i, qtableA[2, 1, 4], error)
                errors.append(error)
                ind.append(i)

                if alpha < 0.001:
                    alpha = 0.001
                alpha *= alpha_dec
            i += 1
        
        if i % 10000 == 0:
            print (i)
    return errors, ind
        

In [ ]:
errr, ind = ce_q_learning()

324 44.099999999999994 44.099999999999994
428 121.08269077169535 76.98269077169536
651 86.63342099813552 34.449269773559834
1004 87.76663028528841 1.1332092871528943
1604 88.1970748752228 0.43044458993438184
1776 88.3728483628216 0.1757734875988035
1918 88.44938641031595 0.07653804749435267
1969 88.48468299424073 0.03529658392477586
2085 88.50182321651798 0.01714022227724854
2124 88.51054459720041 0.00872138068243089
2756 82.16128950001985 6.34925509718056
2854 78.65751697731247 3.5037725227073793
3119 76.65424226000454 2.0032747173079315
3442 73.7510756430314 2.903166616973138
3589 69.38404777447015 4.367027868561252
3594 66.65161529869434 2.732432475775809
4562 63.787024076779396 2.8645912219149423
6206 61.904083105341286 1.88294097143811
6547 60.63884096274188 1.265242142599405
6625 59.77107150368501 0.8677694590568734
6870 58.48756395680214 1.2835075468828663
7091 56.64212365212508 1.8454403046770622
9673 54.56644572953449 2.075677922590586
10000
12070 53.04113340835883 1.525312321

In [19]:
errr

[0,
 0.0,
 0.295849228823891,
 13.27742462273893,
 9.045909395468248,
 6.3608121096114125,
 4.597932115868936,
 3.4050800118051896,
 2.5759802464551598,
 1.803669135785448,
 1.4136687086328408,
 1.0658597701010564,
 0.8589303661498562,
 0.7002611379805472,
 0.5768359284562905,
 0.4298398947088984,
 0.3603058384757247,
 0.30424563484241673,
 0.2585991921574049,
 0.22109496751117064,
 0.19002547526111613,
 0.16409212926422612,
 0.14229647786526556,
 0.11930334316138413,
 0.1042008744015206,
 0.09128726702641643,
 0.07777649023417865,
 0.06849128039336705,
 0.06044732737145608,
 0.05345354566324545,
 0.04735287185872039,
 0.04201538174712738,
 0.03733295178405882,
 0.03266357460113767,
 0.02909439489445731,
 0.025942196089189906,
 0.022567794532520224,
 0.01961142029506391,
 0.017530988509300016,
 0.015681917995777894,
 0.01403646199813835,
 0.012570574407810398,
 0.01126334983604238,
 0.010096559664368954,
 0.009054265770181757,
 0.008122497425908648,
 0.0072889798248283455,
 0.006542904